In [16]:
import os
import json
import time
import csv
import whisper
import pandas as pd
from tqdm import tqdm

In [17]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [18]:
def load_whisper_model(model_size="base"):
    return whisper.load_model(model_size)

In [19]:
def get_files_in_directory(folder_path, file_extension):
    return sorted([f for f in os.listdir(folder_path) if f.endswith(file_extension)])

In [20]:
def load_transcription_data(json_file_path):
    with open(json_file_path, 'r') as file:
        return json.load(file)

In [21]:
def get_matching_transcription(transcription_data, sample_name):
    return next((item['transcription'] for item in transcription_data if item['sample_name'] == sample_name), None)

In [22]:
def transcribe_audio_with_time(model, wav_file_path):
    start_time = time.time()
    result = model.transcribe(wav_file_path)
    end_time = time.time()
    transcription_time = end_time - start_time
    return result['text'], transcription_time

In [23]:
def calculate_cosine_similarity(reference, hypothesis):
    vectorizer = TfidfVectorizer().fit_transform([reference, hypothesis])
    vectors = vectorizer.toarray()
    cos_sim = cosine_similarity(vectors)[0, 1]
    return cos_sim

In [24]:
def write_results_to_csv_pandas(csv_file_path, results):
    df = pd.DataFrame(results, columns=["File", "Whisper Output", "Correct Transcription", "Cosine Similarity", "Transcription Time"])
    df.to_csv(csv_file_path, index=False)

In [64]:
def process_files_and_save_to_csv_pandas(audio_folder_path, transcription_folder_path, model, csv_file_path):
    # Ensure the directory exists for saving CSV
    os.makedirs(os.path.dirname(csv_file_path), exist_ok=True)

    # Get list of audio files and transcription JSON files
    wav_files = get_files_in_directory(audio_folder_path, '.wav')
    json_files = get_files_in_directory(transcription_folder_path, '.json')
    
    results = []

    # Process each transcription JSON file
    for json_file in tqdm(json_files, desc="Processing JSON files", unit="file"):
        # Load the transcription data from JSON
        json_path = os.path.join(transcription_folder_path, json_file)
        transcription_data = load_transcription_data(json_path)
        
        # Extract base name to match with audio files
        base_name = json_file.split("_transcription")[0]  # e.g., "10_words_sample"

        # Check for 5 samples (both normal and with pause)
        for i in range(1, 6):  # Expecting 5 samples
            # Construct the expected WAV filename for both with and without pause
            wav_file_normal = f"{base_name}_{i}.wav"  # e.g., "10_words_sample_1.wav"
            wav_file_with_pause = f"{base_name}_with pause_{i}.wav"  # e.g., "10_words_sample_with pause_1.wav"

            # Process the normal audio file
            wav_path = os.path.join(audio_folder_path, wav_file_normal)
            if os.path.exists(wav_path):
                sample_name = f"sample_{i}"
                correct_transcription = get_matching_transcription(transcription_data, sample_name)
                if correct_transcription:
                    # Transcribe the audio and measure time
                    whisper_output, transcription_time = transcribe_audio_with_time(model, wav_path)
                    cosine_sim = calculate_cosine_similarity(correct_transcription, whisper_output)
                    results.append([wav_file_normal, whisper_output, correct_transcription, cosine_sim, transcription_time])
                    print(f"Processed {wav_file_normal}: Cosine Similarity = {cosine_sim}, Transcription Time = {transcription_time}")
                else:
                    print(f"No matching transcription found for {wav_file_normal} in {json_file}.")

            # Process the audio file with pause
            wav_path_with_pause = os.path.join(audio_folder_path, wav_file_with_pause)
            if os.path.exists(wav_path_with_pause):
                sample_name = f"sample_{i}"
                correct_transcription = get_matching_transcription(transcription_data, sample_name)
                if correct_transcription:
                    # Transcribe the audio and measure time
                    whisper_output, transcription_time = transcribe_audio_with_time(model, wav_path_with_pause)
                    cosine_sim = calculate_cosine_similarity(correct_transcription, whisper_output)
                    results.append([wav_file_with_pause, whisper_output, correct_transcription, cosine_sim, transcription_time])
                    print(f"Processed {wav_file_with_pause}: Cosine Similarity = {cosine_sim}, Transcription Time = {transcription_time}")
                else:
                    print(f"No matching transcription found for {wav_file_with_pause} in {json_file}.")

    # Check if results were collected before writing to CSV
    if results:
        write_results_to_csv_pandas(csv_file_path, results)
        print(f"Results written to {csv_file_path}")
    else:
        print("No results to write to CSV.")

In [53]:
model = load_whisper_model("base")

c:\Conda\envs\whisper\lib\site-packages\whisper\__init__.py:146: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(fp, map_location=device)


In [65]:
audio_folder_path = "../Voices/Male British (Oliver)"
transcription_folder_path = "../Voices/Transcription" 
csv_file_path1 = "./proccessed/Oliver_base.csv"

In [66]:
import warnings
warnings.filterwarnings("ignore", message="FP16 is not supported on CPU; using FP32 instead")

In [67]:
process_files_and_save_to_csv_pandas(audio_folder_path, transcription_folder_path, model, csv_file_path1)

Processing JSON files:   0%|          | 0/10 [00:00<?, ?file/s]

Processed 10_words_sample_1.wav: Cosine Similarity = 0.9999999999999999, Transcription Time = 1.4900169372558594
Processed 10_words_sample_with pause_1.wav: Cosine Similarity = 0.2716186451929929, Transcription Time = 1.7792437076568604
Processed 10_words_sample_2.wav: Cosine Similarity = 0.9999999999999997, Transcription Time = 1.511958360671997
Processed 10_words_sample_with pause_2.wav: Cosine Similarity = 0.37491427238345554, Transcription Time = 1.534898281097412
Processed 10_words_sample_3.wav: Cosine Similarity = 0.9999999999999999, Transcription Time = 1.5458686351776123
Processed 10_words_sample_with pause_3.wav: Cosine Similarity = 0.22576484600261607, Transcription Time = 1.7553083896636963
Processed 10_words_sample_4.wav: Cosine Similarity = 0.9999999999999997, Transcription Time = 1.583765983581543
Processed 10_words_sample_with pause_4.wav: Cosine Similarity = 0.11914719328677731, Transcription Time = 1.6186738014221191
Processed 10_words_sample_5.wav: Cosine Similarity =

Processing JSON files:  10%|█         | 1/10 [00:16<02:25, 16.12s/file]

Processed 10_words_sample_with pause_5.wav: Cosine Similarity = 0.1450404822587595, Transcription Time = 1.649301528930664
Processed 10_words_sample_1.wav: Cosine Similarity = 0.26055567105626243, Transcription Time = 1.5129544734954834
Processed 10_words_sample_with pause_1.wav: Cosine Similarity = 0.9361362641191208, Transcription Time = 1.6535861492156982
Processed 10_words_sample_2.wav: Cosine Similarity = 0.3575475209147722, Transcription Time = 1.5319039821624756
Processed 10_words_sample_with pause_2.wav: Cosine Similarity = 0.9361362641191208, Transcription Time = 1.5967309474945068
Processed 10_words_sample_3.wav: Cosine Similarity = 0.21528271546341884, Transcription Time = 1.636624813079834
Processed 10_words_sample_with pause_3.wav: Cosine Similarity = 0.9266368211041915, Transcription Time = 1.7393519878387451
Processed 10_words_sample_4.wav: Cosine Similarity = 0.11304078416613478, Transcription Time = 1.5628225803375244
Processed 10_words_sample_with pause_4.wav: Cosine 

Processing JSON files:  20%|██        | 2/10 [00:32<02:08, 16.11s/file]

...

Processed 40_words_sample_with pause_5.wav: Cosine Similarity = 0.9032225618759107, Transcription Time = 2.4494540691375732
Processed 50_words_sample_1.wav: Cosine Similarity = 1.0000000000000002, Transcription Time = 2.559157133102417
Processed 50_words_sample_with pause_1.wav: Cosine Similarity = 0.9800376809663689, Transcription Time = 2.6604340076446533
Processed 50_words_sample_2.wav: Cosine Similarity = 1.0000000000000007, Transcription Time = 2.4424710273742676
Processed 50_words_sample_with pause_2.wav: Cosine Similarity = 0.9540892173387038, Transcription Time = 2.554109811782837
Processed 50_words_sample_3.wav: Cosine Similarity = 1.0000000000000002, Transcription Time = 2.509294033050537
Processed 50_words_sample_with pause_3.wav: Cosine Similarity = 0.9236502692895245, Transcription Time = 2.5920727252960205
Processed 50_words_sample_4.wav: Cosine Similarity = 1.0000000000000002, Transcription Time = 2.394598960876465
Processed 50_words_sample_with pause_4.wav: Cosine Simil

Processing JSON files:  90%|█████████ | 9/10 [03:09<00:23, 23.44s/file]

Processed 50_words_sample_with pause_5.wav: Cosine Similarity = 0.7381335090196928, Transcription Time = 2.7227234840393066
Processed 50_words_sample_1.wav: Cosine Similarity = 0.8542408211833338, Transcription Time = 2.5272490978240967
Processed 50_words_sample_with pause_1.wav: Cosine Similarity = 0.8695641616041424, Transcription Time = 2.5671403408050537
Processed 50_words_sample_2.wav: Cosine Similarity = 0.8408180853367483, Transcription Time = 2.4245195388793945
Processed 50_words_sample_with pause_2.wav: Cosine Similarity = 0.8776782349316483, Transcription Time = 2.5551693439483643
Processed 50_words_sample_3.wav: Cosine Similarity = 0.8619161172033042, Transcription Time = 2.526247978210449
Processed 50_words_sample_with pause_3.wav: Cosine Similarity = 0.8830874083542424, Transcription Time = 2.6758480072021484
Processed 50_words_sample_4.wav: Cosine Similarity = 0.8929821067965046, Transcription Time = 2.436497449874878
Processed 50_words_sample_with pause_4.wav: Cosine Sim

Processing JSON files: 100%|██████████| 10/10 [03:34<00:00, 21.48s/file]

Processed 50_words_sample_with pause_5.wav: Cosine Similarity = 0.7032825996613278, Transcription Time = 2.808494806289673
Results written to ./proccessed/Oliver_base.csv
